In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [3]:
%%capture pwd
!pwd

In [4]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
height, width

(300, 500)

# Fig 1

In [15]:
df = (
    pd.read_excel(
        "raw/Figures.xlsx",
        skiprows=1,
        nrows=3,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df = (
    df.set_index("Unnamed: 0")
    .stack()
    .reset_index()
    .set_index(["level_1", "Unnamed: 0"])
    .unstack()[0]
    .reset_index()
)
df.columns = ["year", "morbidly", "obese"]

In [17]:
f = "fig1_obesity"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,year,morbidly,obese
0,1993,0.824392,14.904475
1,1994,1.015056,15.668642
2,1995,0.865891,16.444999
3,1996,0.916274,17.488879
4,1997,1.599698,18.428014


In [23]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:O",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    )
)
area = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    stroke=colors["eco-gray"],
    strokeWidth=0.5,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-pink"], offset=0.9),
        ],
        x1=0.8,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "obese:Q",
        axis=alt.Axis(
            grid=True,
            title="%",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            labelAlign="left",
            ticks=False,
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=0,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
        # scale=alt.Scale(domain=series_domain[serie]),
    )
)
area

alt.Chart(...)